# How to run reV Locally

[reV Gen](https://nrel.github.io/reV/reV/reV.generation.generation.html#reV.generation.generation.Gen) and [reV Econ](https://nrel.github.io/reV/reV/reV.econ.econ.html#reV.econ.econ.Econ) can be run locally using resource .h5 files stored locally or using the [HDF Groups](https://www.hdfgroup.org) Highly Scalable Distributed Service (HSDS) to access resource .h5 file stored in the cloud (currenly on Amazon Web Services -- AWS).

reV use [PySAM](https://pysam.readthedocs.io/en/latest/) to
compute technologically specific capcity factor means and profiles (`Gen`) and mean levelized cost of energy (`Econ`). `reV`
uses ``SAM`` technology terms and input configuration files.


NOTE: In all of these examples, the ``sam_file`` input points to files in
the reV test directory [`TESTDATADIR`](https://github.com/NREL/reV/tree/master/tests/data) that may not be copied in your install. You may want to download the relevant SAM system configs from that directory and point the ``sam_file`` variable to the correct filepath on your computer.

## Running reV Gen+Econ
### Locally

NOTE: as with the `sam_file` path metioned above, the `res_file` path references files in reVs `TESTDATADIR` you might need to download them from [github](https://github.com/NREL/reV/tree/main/tests/data) and point the `res_file` variable to the correct file path on your computer.

#### windpower

Compute wind capacity factors for a given set of latitude and longitude
coordinates:

In [1]:
import os
import numpy as np
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

lat_lons = np.array([[ 41.25, -71.66],
                     [ 41.05, -71.74],
                     [ 41.97, -71.78],
                     [ 41.65, -71.74],
                     [ 41.25, -71.7 ],
                     [ 41.05, -71.78]])

res_file = os.path.join(TESTDATADIR, 'wtk/ri_100_wtk_2012.h5')
sam_file = os.path.join(TESTDATADIR,
                            'SAM/wind_gen_standard_losses_0.json')

pp = ProjectPoints.lat_lon_coords(lat_lons, res_file, sam_file)
gen = Gen.reV_run('windpower', pp, sam_file, res_file,
                    max_workers=1, out_fpath=None,
                    output_request=('cf_mean', 'cf_profile'))
display(gen.out['cf_mean'])
display(gen.out['cf_profile'])

/Users/mrossol/Git_Repos/reV/reV/config/project_points.py:576: UserWarning: WARNING: points are not in sequential order and will be sorted! The original order is being preserved under column "points_order"
  warn(msg)
It appears old SAM v1 keys are being used. Updated key "wind_farm_losses_percent" to "turb_generic_loss".
/Users/mrossol/Git_Repos/reV/reV/SAM/version_checker.py:101: PySAMVersionWarning: It appears old SAM v1 keys are being used. Updated key "wind_farm_losses_percent" to "turb_generic_loss".
  warn(m, PySAMVersionWarning)


array([0.35720587, 0.37741643, 0.500766  , 0.48840794, 0.50351036,
       0.4890139 ], dtype=float32)

array([[0.319, 0.485, 0.6  , 0.543, 0.579, 0.486],
       [0.382, 0.444, 0.288, 0.562, 0.339, 0.601],
       [0.696, 0.398, 0.5  , 0.656, 0.466, 0.677],
       ...,
       [0.833, 0.833, 0.833, 0.833, 0.833, 0.833],
       [0.782, 0.833, 0.833, 0.833, 0.833, 0.833],
       [0.756, 0.833, 0.833, 0.833, 0.833, 0.833]], dtype=float32)

### pvwatts

NOTE: ``pvwattsv5`` and ``pvwattsv7`` are both available from reV.

Compute pv capacity factors and lcoe for all resource gids in Rhode Island:

In [2]:
import os
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

regions = {'Rhode Island': 'state'}

res_file = os.path.join(TESTDATADIR, 'nsrdb/', 'ri_100_nsrdb_2012.h5')
sam_file = os.path.join(TESTDATADIR, 'SAM/naris_pv_1axis_inv13.json')

pp = ProjectPoints.regions(regions, res_file, sam_file)
gen = Gen.reV_run('pvwattsv5', pp, sam_file, res_file,
                    max_workers=1, out_fpath=None,
                    output_request=('cf_mean', 'cf_profile', 'lcoe_fcr'))
display(gen.out['cf_mean'])
display(gen.out['lcoe_fcr'])

array([0.17571239, 0.17621112, 0.1769193 , 0.16443734, 0.16479948,
       0.17848527, 0.17567115, 0.16595586, 0.16590102, 0.17567268,
       0.17551209, 0.17544279, 0.1758483 , 0.1781524 , 0.16609544,
       0.17655854, 0.1766948 , 0.16604662, 0.17523618, 0.17531228,
       0.1666574 , 0.17560133, 0.16629359, 0.17656481, 0.1750054 ,
       0.1752908 , 0.17658393, 0.16675328, 0.16669087, 0.16653797,
       0.16687067, 0.17903888, 0.17576791, 0.17574655, 0.17709686,
       0.17669852, 0.1769204 , 0.17820692, 0.17672373, 0.17549326,
       0.1659799 , 0.1666653 , 0.17583768, 0.17579986, 0.16725487,
       0.16691147, 0.17654173, 0.16690645, 0.16711442, 0.17622866,
       0.17592776, 0.17732163, 0.16735603, 0.1668892 , 0.17714171,
       0.17551471, 0.17758526], dtype=float32)

array([132.45659, 132.08171, 131.55301, 141.5388 , 141.22777, 130.3988 ,
       132.48769, 140.24371, 140.29005, 132.48654, 132.60776, 132.66014,
       132.35422, 130.64244, 140.12585, 131.8218 , 131.72015, 140.16705,
       132.81654, 132.7589 , 139.65335, 132.54037, 139.95888, 131.81712,
       132.9917 , 132.77518, 131.80286, 139.57306, 139.6253 , 139.75351,
       139.47487, 129.99559, 132.41475, 132.43085, 131.4211 , 131.71738,
       131.55219, 130.60246, 131.69858, 132.622  , 140.22339, 139.64673,
       132.36221, 132.39069, 139.1545 , 139.44077, 131.83435, 139.44496,
       139.27144, 132.06856, 132.29443, 131.25452, 139.07037, 139.45938,
       131.38783, 132.60577, 131.05966], dtype=float32)

### Using HSDS

The Highly Scalable Distributed Service (HSDS) is a cloud optimized API to enable access to .h5 files hosted on [AWS](https://registry.opendata.aws). The HSDS software was developed by the [HDF Group](https://www.hdfgroup.org/) and is hosted on Amazon Web Services (AWS) using a combination of EC2 (Elastic Compute) and S3 (Scalable Storage Service). You can read more about the HSDS service [in this slide deck](https://www.slideshare.net/HDFEOS/hdf-cloud-services).


#### Setting up HSDS

To get started install the h5pyd library:

```bash
pip install h5pyd
```

Next, configure h5pyd by running ``hsconfigure`` from the command line, or by
creating a configuration file at ``~/.hscfg``:

```bash
hsconfigure

hs_endpoint = https://developer.nrel.gov/api/hsds
hs_username =
hs_password =
hs_api_key = 3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf
```

**The example API key here is for demonstration and is rate-limited per IP. To
get your own API key, visit https://developer.nrel.gov/signup/**

**Please note that our HSDS service is for demonstration purposes only, if you
would like to use HSDS for production runs of reV please setup your own
service: https://github.com/HDFGroup/hsds and point it to our public HSDS
bucket: s3://nrel-pds-hsds**

#### Using HSDS with reV

Once h5pyd has been installed and configured, [rex](https://github.com/nrel/rex) can pull data directly from AWS using [HSDS](https://github.com/NREL/hsds-examples)

To access the resource data used by reV (NSRDB or WTK) you have to turn on the
``hsds`` flag in the `rex`'s [resource handlers](https://nrel.github.io/rex/rex/rex.renewable_resource.html):

In [ ]:
import rex

nsrdb_file = '/nrel/nsrdb/v3/nsrdb_2013.h5'
with rex.Resource(nsrdb_file, hsds=True) as f:
    meta_data = f.meta
    time_index = f.time_index

display(meta.head())
diplay(time_index)

#### windpower

Compute wind capacity factors for a given set of latitude and longitude
coordinates:

In [8]:
import os
import numpy as np
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

lat_lons = np.array([[ 41.75, -73.66],
                     [ 40.05, -73.74],
                     [ 40.97, -73.78],
                     [ 41.65, -73.74],
                     [ 40.25, -73.7 ],
                     [ 41.05, -73.78]])

res_file = '/nrel/wtk/conus/wtk_conus_2012.h5'  # HSDS 'file' path
sam_file = os.path.join(TESTDATADIR, 'SAM/wind_gen_standard_losses_0.json')

pp = ProjectPoints.lat_lon_coords(lat_lons, res_file, sam_file)
gen = Gen.reV_run('windpower', pp, sam_file, res_file,
                  max_workers=1, out_fpath=None,
                  output_request=('cf_mean', 'cf_profile'))
display(gen.out['cf_profile'])

/Users/mrossol/Git_Repos/reV/reV/config/project_points.py:576: UserWarning: WARNING: points are not in sequential order and will be sorted! The original order is being preserved under column "points_order"
  warn(msg)
/Users/mrossol/Git_Repos/reV/reV/SAM/version_checker.py:101: PySAMVersionWarning: It appears old SAM v1 keys are being used. Updated key "wind_farm_losses_percent" to "turb_generic_loss".
  warn(m, PySAMVersionWarning)


array([[0.06 , 0.213, 0.265, 0.312, 0.83 , 0.827],
       [0.14 , 0.364, 0.438, 0.613, 0.831, 0.833],
       [0.023, 0.685, 0.371, 0.562, 0.833, 0.833],
       ...,
       [0.792, 0.589, 0.833, 0.833, 0.833, 0.833],
       [0.833, 0.833, 0.832, 0.798, 0.833, 0.833],
       [0.711, 0.827, 0.655, 0.19 , 0.833, 0.833]], dtype=float32)

#### pvwatts

NOTE: ``pvwattsv5`` and ``pvwattsv7`` are both available from reV.

Compute pv mean capacity factors and mean levelized cost of energy for all resource gids in New York county New York:

In [4]:
import os
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

regions = {'New York': 'county'}

res_file = '/nrel/nsrdb/v3/nsrdb_2012.h5'  # HSDS 'file' path
sam_file = os.path.join(TESTDATADIR, 'SAM/naris_pv_1axis_inv13.json')

pp = ProjectPoints.regions(regions, res_file, sam_file)
gen = Gen.reV_run('pvwattsv5', pp, sam_file, res_file,
                  max_workers=1, out_fpath=None,
                  output_request=('cf_mean', 'lcoe_fcr'))
display(gen.out['cf_mean'])
display(gen.out['lcoe_fcr'])

## Running Econ from an existing Generation file

While the most efficient way to run reV is to compute generation and econ outputs in SAM at the same time, reV can compute SAM derived Economic outputs from an existing reV Generation output stored in a .h5 file.

In [7]:
import os
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.econ.econ import Econ

cf_file = os.path.join(TESTDATADIR,
                        'gen_out/gen_ri_pv_2012_x000.h5')
sam_files = os.path.join(TESTDATADIR,
                            'SAM/i_lcoe_naris_pv_1axis_inv13.json')

points = slice(0, 100)
econ = Econ.reV_run(points, sam_files, cf_file,
                   year=2012, output_request='lcoe_fcr',
                   max_workers=1, sites_per_worker=25,
                   points_range=None, out_fpath=None)

display(econ.out['lcoe_fcr'])

array([136.10681 , 135.31549 , 131.49303 , 130.75429 , 130.75429 ,
       130.02382 , 125.80684 , 126.49057 , 125.80684 , 125.130455,
       126.49057 , 130.02382 , 125.80684 , 140.20642 , 139.36685 ,
       129.30147 , 131.49303 , 130.75429 , 138.5373  , 138.5373  ,
       130.75429 , 125.130455, 123.799286, 123.799286, 136.90744 ,
       131.49303 , 131.49303 , 130.75429 , 129.30147 , 138.5373  ,
       130.75429 , 130.02382 , 138.5373  , 131.49303 , 133.76015 ,
       125.130455, 131.49303 , 123.799286, 138.5373  , 131.49303 ,
       138.5373  , 125.130455, 123.799286, 130.75429 , 131.49303 ,
       131.49303 , 130.75429 , 137.71754 , 138.5373  , 138.5373  ,
       131.49303 , 129.30147 , 138.5373  , 137.71754 , 128.5871  ,
       130.75429 , 125.80684 , 126.49057 , 126.49057 , 130.75429 ,
       130.02382 , 125.80684 , 126.49057 , 130.02382 , 130.02382 ,
       129.30147 , 133.76015 , 130.02382 , 131.49303 , 138.5373  ,
       123.799286, 138.5373  , 124.4613  , 130.75429 , 130.754